### Setup

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
import time
from tqdm.auto import tqdm, trange
import control
import notebook_setup
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import gym
import torch
import torch.nn as nn
from commonml.helpers.logs import get_tensorboard_scalar_frame

from systems.base import SystemEnv
from systems.plotting import (
    plot_env_response,
    multiple_response_plots
)
from rl import learn_rl, transform_rl_policy, evaluate_rl
from xform import (
    policy_transform,
    pseudo_matrix,
    ab_xform_from_pseudo_matrix,
    basis_vectors,
    is_controllable,
    get_env_samples
)
from mpcontrol import evaluate_mpc

# Systems

### Simple system

\begin{align}
\dot{y_1} &= a \cdot x + b \cdot u(t) \\
&= [a]y + [b] u(t)
\end{align}

In [ ]:
def create_simple(a, b, name='simple'):
    A = np.asarray([[a]], dtype=np.float32)
    B = np.asarray([[b]], dtype=np.float32)
    C = np.eye(1, dtype=np.float32)
    D = np.zeros((1,1), dtype=np.float32)
    return control.ss(A, B, C, D, name=name)

sim = create_simple(-0.1, 1)
sim

In [ ]:
class SimpleEnv(SystemEnv):
    def __init__(self, a, b, q=1, dt=0.01, seed=None):
        self.a = a
        self.b = b
        system = create_simple(a, b)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.period = int((2/np.abs(a)) / dt)
        super().__init__(system, q, dt, seed)
    def reset(self, x=None):
        super().reset(x)
        self.x = (np.asarray(x) or \
             self.random.uniform(
            -1/(2*np.abs(self.a)),
             1/(2*np.abs(self.a)),
             size=(1,)
            )).astype(np.float32)
        return self.x
    def step(self, u: np.ndarray, from_x=None, persist=True):
        x, r, d, i = super().step(u, from_x=from_x, persist=persist)
        return x, r, self.n > self.period, i

In [ ]:
x, u, r = [], [], []
r_ = []
env = SimpleEnv(-0.1, 1)
x.append(env.reset([-1])[0])
for i in range(int(env.period * 5)):
    #u.append(np.sin(i * 2 * np.pi /env.period))
    u.append(-np.sign(x[-1]))
    state, reward, *_ = env.step([u[-1]])
    x.append(state[0])
    r.append(reward)
plt.plot(x, label='x')
# plt.plot(u, label='u')
lines = plt.gca().lines
plt.twinx()
plt.plot(r, label='r', c='g')
plt.legend(handles = plt.gca().lines + lines)
print(sum(r))

### Spring-mass system

$$
m\ddot{x} = -kx -k_f\dot{x} + u(t)
$$

This is a 2nd order ODE. Linearizing it, by letting $y_1 = x, y_2 = \dot{x}$. Then:

\begin{align}
\dot{y}_1 &= 0y_1 + y_2 \\
\dot{y}_2 &= -(k/m)y_1 - (k_f/m)y_2  + u(t)/m
\end{align}

In state space form:

$$
\begin{bmatrix}\dot{y}_1 \\ \dot{y}_2 \end{bmatrix} = 
\begin{bmatrix}0 & 1 \\ -k/m & -k_f/m \end{bmatrix}
\begin{bmatrix}y_1 \\ y_2 \end{bmatrix} + 
\begin{bmatrix}0 \\ 1/m\end{bmatrix}
\begin{bmatrix}u(t)\end{bmatrix}
$$

In [ ]:
def create_spring(k, m, df=0.02, name='spring'):
    A = np.asarray([[0, 1], [-k/m, -df/m]], dtype=np.float32)
    B = np.asarray([[0], [1/m]], dtype=np.float32)
    C = np.eye(2, dtype=np.float32)
    D = np.zeros((2,1), dtype=np.float32)
    return control.ss(A, B, C, D, name=name)

spr = create_spring(1,0.1)
spr

In [ ]:
class SpringMassEnv(SystemEnv):
    def __init__(self, k, m, df=0.02, q=((1,0),(0,1)), dt=0.01, seed=None):
        system = create_spring(k, m, df=df)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(2,), dtype=np.float32)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.period = int(np.pi * 2 * np.sqrt(m / k) / dt)
        super().__init__(system, q, dt, seed)
    def reset(self, x=None):
        super().reset(x)
        self.x = (np.asarray(x) if x is not None else \
                 self.random.uniform([-1, -0.1], [1, 0.1])
                 ).astype(np.float32)
        return self.x
    def step(self, u: np.ndarray, from_x=None, persist=True):
        x, r, d, i = super().step(u, from_x=from_x, persist=persist)
        return x, r, self.n > self.period, i

In [ ]:
x, v, r = [], [], []
env = SpringMassEnv(4,0.2,0.01)
env.reset([-0.5, 0])
for _ in range(int(env.period * 5)):
    action = -1 * np.sign(env.x[1:])
    state, reward, *_ = env.step(action)
    x.append(state[0])
    v.append(state[1])
    r.append(reward)
plt.plot(x, label='x')
plt.plot(v, label='v')
plt.legend()
plt.twinx()
plt.plot(r, label='reward', c='g')
plt.legend()
print(np.sum(r))

### Pendulum system

The state of a pendulum is defined by the angle $\theta$, where the action $u(t)$ is the torque being applied to the pendulum:

\begin{align}
\dot{y_1} &= \dot{\theta} =& y_2 \\
\dot{y_2} &= \ddot{\theta} =& -\frac{g}{l}\sin\theta + \frac{u(t) -k_f\dot{\theta}}{ml^2}
\end{align}

\begin{align}
\begin{bmatrix}\dot{y}_1 \\ \dot{y}_2 \end{bmatrix} = 
\begin{bmatrix}0 & 1 \\ -g\sin(\cdot)/l & -k_f/ml^2 \end{bmatrix}
\begin{bmatrix}y_1 \\ y_2 \end{bmatrix} + 
\begin{bmatrix}0 \\ 1/ml^2\end{bmatrix}
\begin{bmatrix}u(t)\end{bmatrix}
\end{align}

In [ ]:
def create_pendulum(m, l, g, df=0.02, name='pendulum',
                    xformA=np.eye(2, dtype=np.float32),
                    xformB=np.eye(2, dtype=np.float32)):
    g_l = -g / l
    ml2_inv = 1 / (m * l**2)
    def updfcn(t, x, u, params):
        if x.ndim==2:
            x = x.squeeze()
            u = np.atleast_1d(u.squeeze())
        Ax = np.asarray(
            [[0, x[1]],
             [g_l * np.sin(x[0]), -df * x[1] * ml2_inv]],
            dtype=np.float32).sum(axis=1)
        Bu = np.asarray(
            [[0],
             [u[0] * ml2_inv]],
            dtype=np.float32).sum(axis=1)
        return xformA @ Ax + xformB @ Bu
    def outfcn(t, x, u, params):
        return x
    sys = control.NonlinearIOSystem(updfcn, outfcn, name=name,
                                    inputs=1, outputs=2, states=2)
    return sys
pend = create_pendulum(0.1, 1, 10)
pend.linearize(np.asarray([np.pi/2,0]), np.asarray([0]))

In [ ]:
class PendulumEnv(SystemEnv):
    def __init__(
        self, m, l, g, df=0.02, q=((10,0),(0,0.1)), dt=0.01,
        seed=None, xformA=np.eye(2), xformB=np.eye(2)
    ):
        system = create_pendulum(m, l, g, df=df, xformA=xformA, xformB=xformB)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(2,), dtype=np.float32)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.period = int(np.pi * 2 * np.sqrt(l / g) / dt)
        super().__init__(system, q, dt, seed)
    def reset(self, x=None):
        super().reset(x)
        self.x = (np.asarray(x) if x is not None else \
                 self.random.uniform([-np.pi/6, -0.05], [np.pi/6, 0.05])
                 ).astype(np.float32)
        return self.x
    def step(self, u: np.ndarray, from_x=None, persist=True):
        x, r, d, i = super().step(u, from_x=from_x, persist=persist)
        if x.ndim==1:
            x[0] = np.sign(x[0]) * (np.abs(x[0]) % (2 * np.pi))
            if persist:
                self.x[0] = x[0]
        elif x.ndim==2:
            x[0,0] = np.sign(x[0,0]) * (np.abs(x[0,0]) % (2 * np.pi))
            if persist:
                self.x[0,0] = x[0,0]
        return x, r, self.n > self.period, i

In [ ]:
x, v, r = [], [], []
env = PendulumEnv(m=0.1, l=1., g=10, df=0.02, xformA=xformA, xformB=xformB)
env.reset([0.1, 0])
for _ in range(int(env.period * 2)):
    action = -1 * np.sign(env.x[1:])
    state, reward, *_ = env.step(action)
    x.append(state[0])
    v.append(state[1])
    r.append(reward)
plt.plot(x, label='theta')
plt.plot(v, label='theta_dot')
plt.legend()
plt.twinx()
plt.plot(r, label='reward', c='g')
plt.legend()
print(sum(r))

### Cart-pole

The state of an inverted pendulum system is defined by the linear and angular accelerations $\ddot{x}, \ddot{\theta}$. $x$ is positive right, $\theta$ is positive counter-clockwise.

Then, linearizing it by assuming $y_1=\theta, y_2=\dot{\theta}, y_3=x, y_4=\dot{x}$

\begin{align}
\dot{y_1} &= \dot{\theta} =& y_2 \\
\dot{y_2} &= \ddot{\theta} =& \frac{g}{l}\sin y_1 - \frac{k_f y_2}{ml^2} + \frac{\dot{y_4}\cos y_1}{l} \\
\dot{y_3} &= \dot{x} =& y_4 \\
\dot{y_4} &= \ddot{x} =& \frac{m_p \sin y_1\left(g\cos y_1-l\dot{y_1}^2\right) + u(t)}{m_c+m_p-m_p\cos^2 y_1}
\end{align}

In [ ]:
def create_cartpole(mc, mp, l, g, df=0.01, name='cartpole',
                   xformA=np.eye(4), xformB=np.eye(4)):
    g_l = g / l
    ml2_inv = 1 / (mp * l**2)
    def updfcn(t, x, u, params):
        if x.ndim==2:
            # for MPC simulations
            u = np.atleast_1d(u.squeeze())
            x1,x2,x3,x4=x.squeeze() # theta, theta_dot, x, x_dot
        elif x.ndim==1:
            x1,x2,x3,x4=x # theta, theta_dot, x, x_dot
        cx1, sx1 = np.cos(x1), np.sin(x1)
        x4_denom = mc + mp - mp * cx1**2
        # delta x due to state
        x1_ = x2
        x2_ = (g_l * sx1) - (df * ml2_inv * x2) + \
              ((cx1 / l) * ((mp * sx1 * (g*cx1 - l * x2**2)) / x4_denom))
        x3_ = x4
        x4_ = (mp * sx1 * (g*cx1 - l * x2**2)) / x4_denom
        dx_x = np.asarray([x1_, x2_, x3_, x4_], dtype=np.float32)
        # delta x due to action. Factoring out parts in u
        x1_ = 0
        x2_ = (cx1 / l) * (u[0]) / x4_denom
        x3_ = 0
        x4_ = u[0] / x4_denom
        dx_u = np.asarray([x1_, x2_, x3_, x4_], dtype=np.float32)
        return xformA @ dx_x + xformB @ dx_u
    def outfcn(t, x, u, params):
        return x
    sys = control.NonlinearIOSystem(updfcn, outfcn, name=name,
                                    inputs=1, outputs=4, states=4)
    return sys

cart = create_cartpole(10, 1, 1, 10)
cart.linearize(np.asarray([np.pi, 0, 0, 0]), np.asarray([0]))

In [ ]:
class CartpoleEnv(SystemEnv):
    def __init__(self, mc, mp, l, g, df=0.01,
                 q=((1,0,0,0),(0,0.01,0,0),(0,0,0.1,0),(0,0,0,0.01)),
                 dt=0.01, seed=None, xformA=np.eye(4), xformB=np.eye(4)):
        system = create_cartpole(mc, mp, l, g, df=df,
                                 xformA=xformA, xformB=xformB)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(4,), dtype=np.float32)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.period = int(np.pi * 2 * np.sqrt(l / g) / dt)
        super().__init__(system, q, dt, seed)
    def reset(self, x=None):
        super().reset(x)
        self.x = (np.asarray(x) if x is not None else \
                 self.random.uniform(-0.05, 0.05, size=4)
                 ).astype(np.float32)
        return self.x
    def reward(self, xold, u, x):
        return 1.
    def step(self, u: np.ndarray, from_x=None, persist=True):
        x, r, d, i = super().step(u, from_x=from_x, persist=persist)
        if x.ndim==1:
            x[0] = np.sign(x[0]) * (np.abs(x[0]) % (2 * np.pi))
            if persist:
                self.x[0] = x[0]
            done = (np.abs(x[0]) > 0.2095) or \
                   (self.n >= 500) or \
                   (np.abs(x[2]) > 2.4)
        elif x.ndim==2:
            assert not persist, '2 dim array only during MPC simulation'
            x[0,0] = np.sign(x[0,0]) * (np.abs(x[0,0]) % (2 * np.pi))
            done = (np.abs(x[0,0]) > 0.2095) or \
                   (self.n >= 500) or \
                   (np.abs(x[0,2]) > 2.4)
        return x, r, done, i

In [ ]:
th, omega, x, v, r = [], [], [], [], []
env = CartpoleEnv(mc=0.5, mp=0.1, l=1, g=10, df=0.01, dt=0.01)
env.reset([np.pi/90, 0, 0, 0])
done = False
while not done:
# for _ in range(int(5 * env.period)):
    action = -0.5 * np.sign(env.x[0:1])
    state, reward, done, _ = env.step(action)
    th.append(state[0])
    omega.append(state[1])
    x.append(state[2])
    v.append(state[3])
    r.append(reward)
plt.plot(np.asarray(th) * 180 / np.pi, label='theta', c='r')
plt.plot(np.asarray(omega) * 180 / np.pi, label='theta_dot', c='r', ls=':')
lines = plt.gca().lines
plt.ylabel('Angles /deg')
plt.twinx()
plt.plot(x, label='x', c='b')
plt.plot(v, label='x_dot', c='b', ls=':')
plt.plot(r, label='reward')
plt.ylabel('Position')
plt.legend(handles = plt.gca().lines + lines)
print(sum(r))

# Control

## LQR functions

In [ ]:
def evaluate_law(
    systems, x0_fn, q, r=None, ks=None, dt=0.01,
    t_final=10, plot=False, n=10
):
    """
    Compare cost of using LQR trained on one system over others.
    The cost calculated is solely a function of state, and not action.
    If a system is optimized, it should have the control law
    as feedback already.
    """
    c = []
    for i in range(n):
        x0 = x0_fn() if callable(x0_fn) else x0_fn
        costs = []
        for i, s in enumerate(systems):
            if isinstance(s, control.NonlinearIOSystem):
                resp = control.input_output_response(
                    s, np.linspace(0, t_final, int(t_final/dt)), U=0, x0=x0)
            else:
                resp = control.initial_response(
                    s, np.linspace(0, t_final, int(t_final/dt)), x0)
            t, y, x = resp.time, resp.outputs, resp.states
            cost = (x.T @ q @ x).diagonal().sum()
            if ks is not None:
                if isinstance(ks, (list, tuple)):
                    ki = ks[i]
                else:
                    ki = ks
                u = - ki @ x
                cost += (u.T @ r @ u).diagonal().sum()
            costs.append(cost)
        c.append(costs)
    cmean, cstd = np.mean(c, axis=0), np.std(c, axis=0)
    if plot:
        s = [20 if i==0 else 10 for i in range(len(systems))]
        c= ['r' if i==0 else 'b' for i in range(len(systems))]
        plt.scatter(np.arange(i+1), cmean, s=s, c=c)
        plt.ylabel('Cost')
        for j in range(len(systems)):
            plt.text(j, costs[j], '%s:\n%.2f' % (systems[j].name, costs[j]))
        plt.errorbar(np.arange(i+1), cmean, yerr=cstd, ecolor=c)
    return cmean, cstd

In [ ]:
def evaluate_lqr(law: np.ndarray, env, n_eval_episodes=10,
                 clip_actions=True):
    R = []
    start = time.time()
    for e in range(n_eval_episodes):
        x, u, r = [], [], []
        x.append(env.reset())
        done = False
        while not done:
            action = -law @ x[-1]
            if clip_actions:
                action = np.clip(action,
                    a_min=env.action_space.low,
                    a_max=env.action_space.high
                )
            u.append(action)
            state, reward, done, _ = env.step(action)
            r.append(reward)
            if done: break
            x.append(state)
        R.append(sum(r))
    end = time.time()
    runtime = (end - start) / n_eval_episodes
    return np.mean(R), np.std(R), runtime

# Policy transformations

## System specification

In [ ]:
# sys_kwargs = dict(a=-0.1, b=1)
# learn_kwargs = dict(steps=200_000, seed=0, learning_rate=5e-3,
#                     n_steps=4000, batch_size=200, n_epochs=10,
#                     gamma=0.)
# q, r = np.asarray([[1]]), np.asarray([[0.00001]])
# xformA = np.asarray([[0.5]])
# xformB = np.asarray([[-0.5]])
# x0 = np.asarray([-2.5])
# make_env = lambda: SimpleEnv(**sys_kwargs, q=q, seed=0)
# def make_xform_env():
#     env = make_env()
#     env.system.A = xformA @ env.system.A
#     env.system.B = xformB @ env.system.B
#     return env
# sys = create_simple(**sys_kwargs, name='simple')
# env = make_env()

sys_kwargs = dict(k=4, m=0.2, df=0.01)
learn_kwargs = dict(steps=100_000, seed=0, learning_rate=2e-3,
                    n_steps=2048, batch_size=64, n_epochs=10,
                    gamma=0.)
q, r = np.asarray([[1,0], [0,1]]), np.asarray([[0.00001]])
angA, angB = np.pi/4, np.pi
scalarA, scalarB = 0.8, 0.5
xformA = np.asarray([[np.cos(angA), -np.sin(angA)],
                    [np.sin(angA), np.cos(angA)]]).T \
        @ (np.eye(2) * scalarA)
xformB = np.asarray([[np.cos(angB), -np.sin(angB)],
                    [np.sin(angB), np.cos(angB)]]).T \
        @ (np.eye(2) * scalarB)
x0 = np.asarray([-0.5, 0])
make_env = lambda: SpringMassEnv(**sys_kwargs, q=q, seed=0)
def make_xform_env():
    env = make_env()
    env.system.A = xformA @ env.system.A
    env.system.B = xformB @ env.system.B
    return env
env = make_env()
sys = create_spring(**sys_kwargs)

# sys_kwargs = dict(m=0.1, l=1, g=10., df=0.02)
# learn_kwargs = dict(steps=200_000, seed=0, learning_rate=1e-3,
#                     n_steps=2048, batch_size=128, n_epochs=10,
#                     gamma=0.99)
# q, r = np.asarray([[10,0], [0,1e-1]]), np.asarray([[0.00001]])
# angA, angB = np.pi/4, np.pi
# scalarA, scalarB = 0.8, 0.5
# xformA = np.asarray([[np.cos(angA), -np.sin(angA)],
#                     [np.sin(angA), np.cos(angA)]]).T \
#         @ (np.eye(2) * scalarA)
# xformB = np.asarray([[np.cos(angB), -np.sin(angB)],
#                     [np.sin(angB), np.cos(angB)]]).T \
#         @ (np.eye(2) * scalarB)
# x0 = np.asarray([-0.5, 0])
# make_env = lambda: PendulumEnv(**sys_kwargs, q=q, seed=0)
# def make_xform_env():
#     env = PendulumEnv(**sys_kwargs, q=q, seed=0,
#                       xformA=xformA, xformB=xformB)
#     return env
# env = make_env()
# sys = create_pendulum(**sys_kwargs)

# sys_kwargs = dict(mc=0.5, mp=0.1, l=1, g=10, df=0.01)
# learn_kwargs = dict(steps=400_000, seed=0, learning_rate=2e-3,
#                     n_steps=2048, batch_size=64, n_epochs=10,
#                     gamma=0.99)
# q = np.asarray([[1,0,0,0], [0,0.1,0,0],[0,0,1e-5,0],[0,0,0,1e-1]])
# r = np.asarray([[0.00001]])
# xformA = np.diagflat(np.random.RandomState(seed=0).randn(4))
# xformB = np.diagflat(np.random.RandomState(seed=1).randn(4))
# x0 = np.asarray([-np.pi/45, 0, 0, 0])
# make_env = lambda: CartpoleEnv(**sys_kwargs, q=q, seed=0)
# def make_xform_env():
#     env = CartpoleEnv(**sys_kwargs, q=q, seed=0,
#                       xformA=xformA, xformB=xformB)
#     return env
# env = make_env()
# sys = create_cartpole(**sys_kwargs)

# sys_kwargs = dict(vp=vp, sp=sp)
# learn_kwargs = dict(steps=200_000, seed=0, learning_rate=2e-4, n_steps=2000,
#                    gamma=0.8)
# q = np.diagflat([1,1,1,0.1,0.1,0.1,1,1,1,0.1,0.1,0.1])
# r = np.eye(4) * 1e-4
# xformA = np.random.RandomState(seed=0).randn(12,12)
# xformB = np.diagflat(np.random.RandomState(seed=1).randn(4))
# x0 = np.zeros(12, dtype=np.float32)
# make_env = lambda: MultirotorEnv(**sys_kwargs, seed=0)
# def make_xform_env():
#     env = MultirotorEnv(**sys_kwargs, q=q, seed=0,
#                       xformA=xformA, xformB=xformB)
#     return env
# env = make_env()
# sys = create_multirotor(**sys_kwargs)

## Linear Dynamics

### Evaluation w/ LQR

In [ ]:
env = make_env()
env_ = make_xform_env()
if not isinstance(env.system, control.LinearIOSystem):
    warnings.warn(('Must be a linear system in StateSpace from. '
                   'Use `control.linearize()` to convert.'))
    sys = env.system.linearize(
            np.zeros(env.system.nstates), np.zeros(env.system.ninputs))
    sys_xform = env_.system.linearize(
            np.zeros(env_.system.nstates), np.zeros(env_.system.ninputs))
else:
    sys = env.system
    sys_xform = env_.system

# Linear transformation with
k_og, *_ = control.lqr(sys, q, r)
sys_opt = sys.feedback(k_og)
sys_opt.name = 'sys_opt'

# transformed system but with old control law
sys_xform_old = sys_xform.feedback(k_og)
sys_xform_old.name = 'sys_xform_old'

# Optimizing on modified system
k, *_ = control.lqr(sys_xform, q, r)
sys_xform_opt = sys_xform.feedback(k)
sys_xform_opt.name = 'sys_xform_opt'

# Optimizing using transformed law
feedback = policy_transform(sys, xformA, xformB, k_og)
sys_xform_opt2 = sys_xform.feedback(feedback)
sys_xform_opt2.name = 'sys_xform_opt2'

In [ ]:
# print(evaluate_lqr(k_og, make_env()))
print(evaluate_lqr(k_og, make_xform_env()))
print(evaluate_lqr(k, make_xform_env()))
print(evaluate_lqr(feedback, make_xform_env()))

In [ ]:
multiple_response_plots([
    'old policy on old system',
    lambda: plot_env_response(make_env(), x0, k_og),
    'old policy on new system',
    lambda: plot_env_response(make_xform_env(), x0, k_og),
    'Optimal LQR on new system',
    lambda: plot_env_response(make_xform_env(), x0, k),
    'old policy transformation on new system',
    lambda: plot_env_response(make_xform_env(), x0, feedback)
])

### Evaluation w/ MPC

In [ ]:
env = make_env()
env_ = make_xform_env()
if not isinstance(env.system, control.LinearIOSystem):
    warnings.warn(('Must be a linear system in StateSpace from. '
                   'Use `control.linearize()` to convert.'))
    sys = env.system.linearize(
            np.zeros(env.system.nstates), np.zeros(env.system.ninputs))
    sys_xform = env_.system.linearize(
            np.zeros(env_.system.nstates), np.zeros(env_.system.ninputs))
else:
    sys = env.system
    sys_xform = env_.system
state_xform, action_xform = policy_transform(sys, xformA, xformB)

In [ ]:
sys_xform

In [ ]:
# print(evaluate_mpc(make_env(), horizon=5, model_env=make_env()))
print(evaluate_mpc(make_xform_env(), horizon=10, model_env=make_env()))
print(evaluate_mpc(make_xform_env(), horizon=10, model_env=make_xform_env()))
print(evaluate_mpc(make_xform_env(), horizon=10, model_env=make_env(), state_xform=state_xform, action_xform=action_xform))

## Non-linear Dynamics

In [ ]:
def policy_transfrom_nonlinear(sys, xformA=None, xformB=None, ctrl_law=None):
    F_A = (lambda x: F_A @ x) if not callable(xformA) else xformA
    if isinstance(xformB, InvertibleFunction):
        F_B_ = xformB.invert()
    else:
        F_B_ = np.linalg.pinv(F_B)
    if isinstance(B, InvertibleFunction):
        B_ = B.invert()
    
    if ctrl_law is not None:
        def func(x):
            return B_(F_B_((F_A(A@x) - A@x))) + \
                   B_(F_B_(B@x))
        return InvertibleFunction(func, None)
    else:
        return \
            InvertibleFunction((lambda x: -B_(F_B_((F_A(A@x) - A@x)))), None), \
            InvertibleFunction((lambda x: B_(F_B_(B@x))), None)

In [ ]:
env = make_env()
env_ = make_nonlinear_xform_env()
if not isinstance(env.system, control.LinearIOSystem):
    warnings.warn('Must be a linear system in StateSpace from. Use `control.linearize()` to convert.')
    sys = env.system.linearize(
            np.zeros(env.system.nstates), np.zeros(env.system.ninputs))
    sys_xform = env.system.linearize(
            np.zeros(env_.system.nstates), np.zeros(env_.system.ninputs))
else:
    sys = env.system
    sys_xform = env_.system

# Linear transformation with
k_og, *_ = control.lqr(sys, q, r)
sys_opt = sys.feedback(k_og)
sys_opt.name = 'sys_opt'

# transformed system but with old control law
sys_xform_old = sys_xform.feedback(k_og)
sys_xform_old.name = 'sys_xform_old'

# Optimizing on modified system
k, *_ = control.lqr(sys_xform, q, r)
sys_xform_opt = sys_xform.feedback(k)
sys_xform_opt.name = 'sys_xform_opt'

# Optimizing using transformed law
feedback = policy_transform(sys, xformA, xformB, k_og)
sys_xform_opt2 = sys_xform.feedback(feedback)
sys_xform_opt2.name = 'sys_xform_opt2'

In [ ]:
# print(evaluate_law(k_og, make_env()))
print(evaluate_law(k_og, make_xform_env()))
print(evaluate_law(k, make_xform_env()))
print(evaluate_law(feedback, make_xform_env()))

In [ ]:
multiple_response_plots([
#     # Cost of using old policy on old system
#     lambda: plot_env_response(make_env(), x0, k_og),
    # Cost of using old policy on new system
    lambda: plot_env_response(make_xform_env(), x0, k_og),
    # Optimal LQR cost learned on new system
    lambda: plot_env_response(make_xform_env(), x0, k),
    # Cost after old policy transformation on new system
    lambda: plot_env_response(make_xform_env(), x0, feedback)
])

## Data-driven

In [ ]:
# experiment config

# Whether the knowledge of the source system is known,
# or approximated from sampled experiences
data_driven_source = True
# Whether to assume that the system transformations are known
# and not approximate
accurate_xfer = False
# The factor by which action bounds are relaxed to fully allow the
# transformed policy to interact with environment
constrained_actions = None # True by default, not implemented yet
buffer_episodes=5
name = env.__class__.__name__
if data_driven_source and not accurate_xfer:
    name += 'StochasticAll'
elif data_driven_source:
    name += 'StochasticSource'
elif not accurate_xfer:
    name += 'StochasticXfer'

In [ ]:
# train rl policy on original environment
agent = learn_rl(make_env(), tensorboard_log=name+'/Source',
                 **learn_kwargs)

In [ ]:
plot_env_response(make_env(), x0, agent)

In [ ]:
env = make_env()
env_ = make_xform_env()
# linearize a non-linear system to get A,B,C,D representation,
# and the resulting pseudo matrix P_s of the source task
if isinstance(env.system, control.LinearIOSystem):
    _sys_linear = env.system
else:
    _sys_linear = env.system.linearize(x0 * 0, env.action_space.sample() * 0)
# get the pseudo matrix representing source system dynamics
if data_driven_source:
    xu, x = get_env_samples(env, 1, agent)
    P_s = (x @ xu.T) @ np.linalg.pinv(xu @ xu.T)
else:
    P_s = pseudo_matrix(_sys_linear, env.dt)
del _sys_linear
# get pseudo matrix representing target system dynamics
xu, x = get_env_samples(env_, buffer_episodes, agent)
P_t = (x @ xu.T) @ np.linalg.pinv(xu @ xu.T)
# get the relationship between source and target systems
A_s, B_s, A_t, B_t, F_A, F_B = ab_xform_from_pseudo_matrix(P_s, P_t, env.dt)
C_s, D_s = np.eye(len(A_s)), np.zeros_like(B_s)
if accurate_xfer:
    F_A, F_B = xformA, xformB
# generate policy transforms from the source system,
# and its relationship to the target system
source_system = control.ss(A_s, B_s, C_s, D_s)
state_xform, action_xform = policy_transform(source_system, F_A, F_B)

In [ ]:
print('State error:', np.linalg.norm(xformA-F_A))
print('Action error:', np.linalg.norm(xformB-F_B))

In [ ]:
# fine-tine source policy on target environment
agent_new = learn_rl(make_xform_env(),
                     reuse_parameters_of=agent,
                     tensorboard_log=name+'/Tuned', **learn_kwargs)

In [ ]:
# append a transformation to source policy
agent_xform = transform_rl_policy(agent, state_xform, action_xform)

In [ ]:
# fine-tine the transformed policy, except xforms
agent_xform_tuned = learn_rl(
    make_xform_env(),
    reuse_parameters_of=agent_xform,
    learnable_transformation=False,
    tensorboard_log=name+'/XformedTuned', **learn_kwargs
)
print('state_xform', agent_xform_tuned.policy.state_xform)
print('action_xform', agent_xform_tuned.policy.action_xform)

In [ ]:
# fine-tine the transformed policy, including xforms
agent_xform_tuned_all = learn_rl(
    make_xform_env(),
    reuse_parameters_of=agent_xform,
    learnable_transformation=True,
    tensorboard_log=name+'/XformedTunedAll', **learn_kwargs
)
print('state_xform', agent_xform_tuned_all.policy.state_xform.data)
print('action_xform', agent_xform_tuned_all.policy.action_xform.data)

In [ ]:
print('Source policy on source task')
print(evaluate_rl(agent, make_env(), n_eval_episodes=10))
print('Reusing source policy')
print(evaluate_rl(agent, make_xform_env(), n_eval_episodes=10))
print('Tuning source policy')
print(evaluate_rl(agent_new, make_xform_env(), n_eval_episodes=10))
print('Transforming source policy')
print(evaluate_rl(agent_xform, make_xform_env(), n_eval_episodes=10))
print('Tuning transformed policy, except for transformations')
print(evaluate_rl(agent_xform_tuned, make_xform_env(), n_eval_episodes=10))
print('Tuning transformed policy, including transformations')
print(evaluate_rl(agent_xform_tuned_all, make_xform_env(), n_eval_episodes=10))

### Plots

In [ ]:
multiple_response_plots([
#     r'$\pi_s$ on $P_s$ ',
#     lambda: plot_env_response(make_env(), x0, agent),
    r'$\pi_s$ on $P_t$ ',
    lambda: plot_env_response(make_xform_env(), x0, agent),
    r'$\pi_s^*$ on $P_t$ ',
    lambda: plot_env_response(make_xform_env(), x0, agent_new, legend=False),
    r'$\pi_t$ on $P_t$ ',
    lambda: plot_env_response(make_xform_env(), x0, agent_xform, legend=False),
    r'$\pi_t^-$ on $P_t$ ',
    lambda: plot_env_response(make_xform_env(), x0, agent_xform_tuned, legend=False),
    r'$\pi_t^*$ on $P_t$ ',
    lambda: plot_env_response(make_xform_env(), x0, agent_xform_tuned_all, legend=False)
], figsize=(6,8))

In [ ]:
# remember to specify up-to-date directory
# name = env.__class__.__name__
df = get_tensorboard_scalar_frame('tensorboard/%s/Source_1' % name)
dft = get_tensorboard_scalar_frame('tensorboard/%s/Tuned_1' % name)
dfxt = get_tensorboard_scalar_frame('tensorboard/%s/XformedTuned_1' % name)
dfxta = get_tensorboard_scalar_frame('tensorboard/%s/XformedTunedAll_1' % name)

%matplotlib inline
last_tstep = df.index[-1]
plt.figure(figsize=(6,2))
for i, (frame, label) in enumerate([
    (df, '$\pi_s$ on $P_s$'),
    (dft, '$\pi_s^*$ on $P_t$'),
    (dfxt, '$\pi_t^-$ on $P_t$'),
    (dfxta, '$\pi_t^+$ on $P_t$')
]):
    if i > 0:
        frame.index = frame.index + last_tstep
    plt.plot(frame['rollout', 'ep_rew_mean'], label=label)
if name.startswith('Simp'):
    plt.legend()
plt.ylabel('Mean episodic reward')
plt.xlabel('Learning time steps')
plt.setp(plt.xticks()[1], rotation=15)
plt.grid(True, 'both')

# Task similarities

Ultimately, task similarity is measuring the performance difference between two tasks in terms of the cost/reward function and the task dynamics. So if two tasks have similar reward functions (same domain, range, mapping), regardless of the nature of state/action space, optimal control function for one task will be optimal for another task.

- If kernel of the representative matrices are identical -> same stability points -> tasks similar?
    - i.e. if basis vectors of kernels are similar (via dot product)

## Initial state transformations

Given a domain of states, how does the matrix $A$ of different systems map them to a different range?

In [ ]:
pend_states = np.asarray([[-np.pi/6, -0.05],
                          [-np.pi/6, 0.05],
                          [np.pi/6, 0.05],
                          [np.pi/6, -0.05]]).T

In [ ]:
# See how the dynamics matrix A transforms state into derivative
from matplotlib.patches import Polygon
cm = plt.cm.viridis
plt.figure(figsize=(10,10))
plt.gca().add_patch(Polygon(pend_states.T, alpha=0.8, label='OG', ec='k', color=cm(0.5)))
for i, sys in enumerate(linear_pend[5:]):
    plt.gca().add_patch(Polygon(
        (pend_states + (sys.A @ pend_states) * 1e-1).T,
        ec='k', color=cm(i/len(linear_pend)), alpha=0.2, label='%d: %.2f' % (i, np.linalg.det(sys.A))
    ))
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.xlabel(r'$\dot{\theta}$')
plt.ylabel(r'$\partial\theta/\partial t$')
plt.legend()

## State transition matrices

In [ ]:
x = np.linspace(-5,5, 5)
y = np.linspace(-5,5, 5)
xgrid, ygrid = np.meshgrid(x,y)

In [ ]:
%matplotlib notebook
zgrid = xgrid + ygrid
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot3D([min(x), max(x)], [0,0], [0,0], lw=2, c='k')
ax.plot3D([0,0], [min(y),max(y)], [0,0], lw=2, c='k')
ax.plot3D([0,0], [0,0], [np.min(zgrid),np.max(zgrid)], lw=2, c='k')
ax.set_xlabel('a')
ax.set_ylabel('$\epsilon$')
ax.set_zlabel('$a^*$')
surf = ax.plot_surface(xgrid, ygrid, zgrid, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, alpha=0.4)

In [ ]:
%matplotlib notebook
zgrid = xgrid * ygrid
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot3D([min(x), max(x)], [0,0], [0,0], lw=2, c='k')
ax.plot3D([0,0], [min(y),max(y)], [0,0], lw=2, c='k')
ax.plot3D([0,0], [0,0], [np.min(zgrid),np.max(zgrid)], lw=2, c='k')
ax.set_xlabel('a')
ax.set_ylabel('$\epsilon$')
ax.set_zlabel('$a^*$')
colors = cm.RdYlGn((np.sign(zgrid) == np.sign(xgrid)).astype(np.float32).T)
ax.plot_surface(xgrid, ygrid, np.sign(zgrid) == np.sign(xgrid),
                       facecolors=colors, linewidth=0, antialiased=False, alpha=0.6, rstride=1, cstride=1)
surf = ax.plot_surface(xgrid, ygrid, zgrid, cmap=None,
                       linewidth=0, antialiased=False, alpha=0.4)

## Cost surfaces

Assuming a similar cost function, but different state reachabilities, can the alignment between cost surfaces be a measure of task similarity?